# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Jared Vitug
- Miguel Morales
- Phat Ly
- Kevin Mach

# Group Members IDs

- A92083122
- A########
- A########
- A12647584

# Research Question

Can we predict a current winning nba bracket from regular season matchup statistics collected over the last 10 years?
In a best-of-seven NBA playoff series, two teams will matchup against each other consecutively at least four times. This is unlike the regular season schedule in any way, where teams in the same division play at most four times but over the span of an 82 game season. However, we believe that from regular season statistics gathered from the eight current playoff matchups, over the span of the last ten years, we can predict an accurate playoff bracket for the current season’s playoffs. We will look at data such as each team’s winning percentage, number of assists, total offensive rebounds, number of roster changes, etc. to create a model that will tell us which team will win in a playoff series. We will take that model and use it to pick the winners of our bracket.


## Background and Prior Work

With NBA playoffs in full effect, we knew we wanted to research something relative to basketball that was also fun.  We all know sports are near impossible to predict and basketball is no exception - or is it? We began to wonder what if, given a playoff bracket, we could predict each round of the playoffs and thereby figure out who would win the championship.

Each of us watches basketball and knows a few plays can define a game.  Injuries happen, human error occurs, and upsets take place.  However, in the world of sports betting, none of this matters - only numbers matter.  That’s why we decided to find the best combination of regular season matchup stats (between two teams pitted against each other in the playoffs) that can successfully predict a series outcome.

When it comes to the individual statistics, we know that the team which wins the turnover battle usually comes out on top, as does the team with more rebounds and shooting percentage.  However, with all the different play-styles in today’s NBA (iso-ball, backcourt dominance, frontcourt dominance) we want to find out if there are other team stats that can help us predict winners, such as three pointers taken, team fouls, timeouts used, etc.


https://towardsdatascience.com/predicting-nba-winning-percentage-in-upcoming-season-using-linear-regression-f8687d9c0418
https://github.com/COGS108/FinalProjects-Wi18/blob/master/001-FinalProject.ipynb
The above link is to a similar project where someone took 40 years of basketball data from basketball-reference.com and trained the data to predict NBA winning percentage.  The article says that “average age of the players, margin of victory, number of points scored, number of returning players, and number of blocks” were some of the statistics included, which gives us some idea as to which statistics to pull.   The study’s analysis discusses how wrong the model was in over-predicting winning seasons and underpredicting losing seasons with the reason being injuries, trades, and retirements - all things which are hard to predict and difficult to assess from a mathematical perspective.

# Hypothesis


We hypothesize that it is possible to predict the winner of a 7-game playoff series between two NBA teams by analyzing their regular season matchup statistics.
By narrowing our data to historical team matchups, we can examine specific factors that may be pertinent in explaining why one team may have the competitive edge. There are many accessible statistics for every NBA game, and this data can be utilized to determine if one franchise may have a playstyle better suited for a particular opponent. Features such as the record of the matchups (giving more weight to recent wins), number of possessions, offensive rebounds, total assists, turnovers, etc. are all telling of how advantageous a team is over the other. Although teams change over time due to player, coach, and management contracts, we will attempt to take those conditions into account by feature engineering, as winning teams tend to have less structure change. Additionally, we can favor more recent data as it will be more telling of the current matchup. Therefore, with a catalog of in depth matchup statistics, we hypothesize that we can create a strong dataset for our model to predict a correct playoff bracket.


# Dataset(s)

Dataset Name: NBA Advanced Stat
Dataset Source : https://stats.nba.com/
Dataset Link  : https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1&Season=2018-19&SeasonType=Regular%20Season  	
Number of datasets planned : 10 total, 1 for each regular season starting from August 2008 -  April 2019
 
Dataset Explanation: This website contains datasets about the NBA teams’ statistics from year 1999 to present date. Those datasets are organized as in form of a table, where the rows represent the teams and the columns are observations such as win-loss record, shooting percentages, rebounds ..etc.  Since we are planning on using the statistic of the past 10 regular season matchups, this website is useful in that it contains the information from the previous years and we will have to select the options provided in the website (in the provided link) to get the optimal dataset for our project. We will try to pull the regular season information for each year (from 2008 to present date), then drop off teams that are not participating in the current year playoff. We might also need to differentiate each season statistics, perhaps giving more weight to recent year stats than older years.  


# Setup

In [1]:
# Imports 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen
#!sudo pip install lxml



In [2]:
#this code creates a list of urls for us to create our dataframes from

#example URL
# https://www.basketball-reference.com/teams/MIL/2019/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8

year_array = ["2010", "2011", "2012", "2013", "2014", "2015", 
              "2016", "2017", "2018", "2019"]
team_array = ["GSW", "HOU", "POR", "DEN", "LAC", "UTA", "OKC", 
             "SAS", "MIL", "BOS", "PHI", "TOR", "DET", "IND", "ORL", "BRK"]

def create_url(year_array, team_array):
    #variables for appending the URLs
    begLink = "https://www.basketball-reference.com/teams/"
    endLink = "/gamelog/?fbclid=IwAR3BFW5ivLDuQE5NRVkPbHnEIlwe-CCCsoeo8RxOxmcPHssP0_mzfJgsVr8"
    #array to store the url's
    #this array has '1' as parameter because numpy arrays
    #need some value to store - keep this in mind that the first
    #index with an actual url is index 1 (through 160)
    url_list = []
    #loop through team_array 
    for i in team_array:
        #loop through each year in 
        for j in year_array:
            team_name = i
            # if the team is Brooklyn and the year is 2010-2012, team name
            # is New Jersey Nets
            if(i == "BRK" and (j == "2010" or j == "2011" or j == "2012")):
                team_name = "NJN"    
            temp_url = begLink + team_name + "/" + j + endLink
            #print(temp_url)
            url_list.append(temp_url)
    #print(url_list)
    return url_list

            
url_list = create_url(year_array, team_array)
#for url in url_list:
    #print(url)

In [3]:
#this code creates a list of dataframes corresponding to the url's in url_list

#list to store individual dataframes before combining
df_list = []
#i = 0
for url in url_list:
    #print(i)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser') #get contents of webpage
    nbatables = soup.findAll("table", 'row_summable sortable stats_table') #get tables
    tbl1 = nbatables[0]
    new_tbl1 = pd.DataFrame(columns=range(0,40), index = range(0,91))
    
    #get the column names for our first table
    ind=0
    cols_list = []
    for header in tbl1.find_all('tr'): #specify HTML tags
        header_name = header.find_all('th') #tag containing column names
        for head in header_name:
            cols_list.append(head.get_text()) #get the text from between the tags
    
    #fill in contents for each table
    row_marker = -1
    for row in tbl1.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td') # different tag than above for table contents
        for column in columns:
            new_tbl1.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1
    df_list.append(new_tbl1)
    #i = i + 1


In [4]:
def create_team_df(team, df_list):
    #team_df = df_list[0]
    if(team == "GSW"):
        i = 0
        team_df = df_list[i];
        while i < 9:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "HOU"):
        i = 10
        team_df = df_list[i];
        while i < 19:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "POR"):
        i = 20
        team_df = df_list[i];
        while i < 29:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "DEN"):
        i = 30
        team_df = df_list[i];
        while i < 39:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "LAC"):
        i = 40
        team_df = df_list[i];
        while i < 49:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "UTA"):
        i = 50
        team_df = df_list[i];
        while i < 59:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "OKC"):
        i = 60
        team_df = df_list[i];
        while i < 69:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "SAS"):
        i = 70
        team_df = df_list[i];
        while i < 79:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "MIL"):
        i = 80
        team_df = df_list[i];
        while i < 89:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "BOS"):
        i = 90
        team_df = df_list[i];
        while i < 99:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "PHI"):
        i = 100
        team_df = df_list[i];
        while i < 109:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "TOR"):
        i = 110
        team_df = df_list[i];
        while i < 119:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "DET"):
        i = 120
        team_df = df_list[i];
        while i < 129:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "IND"):
        i = 130
        team_df = df_list[i];
        while i < 139:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "ORL"):
        i = 140
        team_df = df_list[i];
        while i < 149:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    if(team == "BRK"):
        i = 150
        team_df = df_list[i];
        while i < 159:
            team_df = team_df.append(df_list[i+1])
            i = i + 1
    #NOTE: renamed opponent points to OppP
    team_df.columns = [
 'G',
 'Date',
 '\xa0',
 'Opp',
 'W/L',
 'Tm',
 'OppP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 '\xa0',
 'oppFG',
 'oppFGA',
 'oppFG%',
 'opp3P',
 'opp3PA',
 'opp3P%',
 'oppFT',
 'oppFTA',
 'oppFT%',
 'oppORB',
 'oppTRB',
 'oppAST',
 'oppSTL',
 'oppBLK',
 'oppTOV',
 'oppPF']
    return team_df
    

In [6]:
#create our team dataframes PRE cleaning

GSW_df = create_team_df("GSW", df_list)
HOU_df = create_team_df("HOU", df_list)
POR_df = create_team_df("POR", df_list)
DEN_df = create_team_df("DEN", df_list)
LAC_df = create_team_df("LAC", df_list)
UTA_df = create_team_df("UTA", df_list)
OKC_df = create_team_df("OKC", df_list)
SAS_df = create_team_df("SAS", df_list)
MIL_df = create_team_df("MIL", df_list)
BOS_df = create_team_df("BOS", df_list)
PHI_df = create_team_df("PHI", df_list)
TOR_df = create_team_df("TOR", df_list)
DET_df = create_team_df("DET", df_list)
IND_df = create_team_df("IND", df_list)
ORL_df = create_team_df("ORL", df_list)
BRK_df = create_team_df("BRK", df_list)

GSW_df

,G,Date,,Opp,W/L,Tm,OppP,FG,FGA,FG%,...,oppFT,oppFTA,oppFT%,oppORB,oppTRB,oppAST,oppSTL,oppBLK,oppTOV,oppPF
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2009-10-28,,HOU,L,107,108,44,89,.494,...,16,20,.800,13,41,30,14,7,18,22
2,2,2009-10-30,@,PHO,L,101,123,36,85,.424,...,21,34,.618,9,47,30,7,7,23,25
3,3,2009-11-04,,MEM,W,113,105,47,87,.540,...,24,36,.667,13,46,24,8,4,17,20
4,4,2009-11-06,,LAC,L,90,118,29,84,.345,...,24,32,.750,7,44,28,11,9,16,29
5,5,2009-11-08,@,SAC,L,107,120,42,83,.506,...,29,38,.763,15,52,21,8,4,14,22
6,6,2009-11-09,,MIN,W,146,105,52,91,.571,...,16,20,.800,21,53,23,7,6,28,31
7,7,2009-11-11,@,IND,L,94,108,37,92,.402,...,25,34,.735,16,57,18,5,12,18,25
8,8,2009-11-13,@,NYK,W,121,107,49,84,.583,...,15,19,.789,12,37,21,7,4,21,21
9,9,2009-11-14,@,MIL,L,125,129,46,82,.561,...,19,30,.633,15,46,21,5,4,14,22


# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*